In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from utils.coords import *
from analysis.peaksdata import *

In [ ]:
# font format for figures
# see: https://stackoverflow.com/questions/33955900/matplotlib-times-new-roman-appears-bold
del mpl.font_manager.weight_dict['roman']
mpl.font_manager._rebuild()

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 12
plt.rcParams["font.weight"] = 'normal'
plt.rcParams['pdf.fonttype'] = 42 #TrueType

In [ ]:
regions = []
regions.append(('Alps', 'alps', [45.8325,  6.8650]))
regions.append(('Sahara', 'sahara', [30.3801, 8.6932]))
regions.append(('Andes-Chile', 'andes_chile', [-21.4483, -68.0708]))
regions.append(('Karakoram', 'karakoram', [35.8283, 76.4608]))
regions.append(('Rockies-Colorado', 'colorado', [39.0782,-106.6986]))
regions.append(('Li-River', 'yangshuo', [24.9917, 110.4617]))
regions.append(('Himalayas', 'himalaya', [28.6683, 84.3004]))
regions.append(('Norway', 'norway', [62.1167, 6.8075]))
regions.append(('Alaska', 'alaska', [62.9500, -151.0908]))
regions.append(('Patagonia', 'patagonia', [-50.8925, -73.1533]))
regions.append(('Andes-Peru', 'andes_peru', [-9.0874, -77.5737]))
regions.append(('Rockies-Canada', 'rockies', [50.8703, -115.6517]))
regions.append(('Appalachians', 'appalachians', [35.0355, -83.5380]))
regions.append(('Scottish-Highlands', 'highlands', [57.0667, -3.3917]))
regions.append(('Gobi', 'gobi', [39.9608, 86.3917]))
regions.append(('Pyrenees', 'pyrenees', [42.589324, 0.937731]))

In [ ]:
diskRadius = 30   # km

for r in regions[:]:
    
    regionFullName, regionName, filterCoords = r
    peaksFile = 'data/regionPeaks/%s.csv' % regionName

    # filter peaks
    df = pd.read_csv(peaksFile)    
    peakDist = np.zeros((df.shape[0],))
    for i in range(df.shape[0]):
        peakDist[i] = distance_haversine((filterCoords[0],filterCoords[1]), (df['latitude'][i], df['longitude'][i]))    
    df = df[peakDist <= diskRadius]
    
    numPeaks = df.shape[0]
    print(regionFullName, numPeaks)
    
    # add extra columns to the dataframe
    df = addExtraColumns(df)
    
    # compute distributions
    distributions = computeDistributions(df, diskRadius=diskRadius)
    
    fig = plt.figure(figsize=(16,16))
    fig.subplots_adjust(wspace=0.1)
    
    barColor  = (216/255, 226/255, 238/255, 1.0)
    edgeColor = (137/255, 151/255, 168/255, 1.0)
    
    ax = fig.add_subplot(6, 6, 1)
    ax.set_ylabel('%s\n%d peaks\n(%.4f, %.4f)\n' % (regionFullName, df.shape[0], filterCoords[0], filterCoords[1]), 
                  multialignment='center', fontsize=18)
    h = ax.bar(distributions['elevRel']['x'],
               distributions['elevRel']['hist'] / distributions['elevRel']['hist'].sum(), 
               width=np.diff(distributions['elevRel']['bins']), color=barColor, edgecolor=edgeColor)
    ax.set_ylim([0, 1.0])
    ax.set_xticks([0.0, 0.5, 1.0])
    ax.set_title('norm. elevation')

    ax = fig.add_subplot(6, 6, 2)
    h = ax.bar(distributions['prominence']['x'],
               distributions['prominence']['hist'] / distributions['prominence']['hist'].sum(), 
               width=np.diff(distributions['prominence']['bins']), color=barColor, edgecolor=edgeColor)
    ax.set_ylim([0, 1.0])
    ax.set_yticklabels([])
    ax.set_xticks([0, 1000, 2000])
    ax.set_title('prominence (m)')

    ax = fig.add_subplot(6, 6, 3)
    h = ax.bar(distributions['domGroup']['x'],
               distributions['domGroup']['hist'] / distributions['domGroup']['hist'].sum(), 
               width=np.diff(distributions['domGroup']['bins']), color=barColor, edgecolor=edgeColor)
    ax.set_ylim([0, 1.0])
    ax.set_yticklabels([])
    ax.set_xticks([0.0, 0.5, 1.0])
    ax.set_title('dominance groups')

    ax = fig.add_subplot(6, 6, 4)
    h = ax.bar(distributions['isolation']['x'],
               distributions['isolation']['hist'] / distributions['isolation']['hist'].sum(), 
               width=np.diff(distributions['isolation']['bins']), color=barColor, edgecolor=edgeColor)
    ax.set_ylim([0, 1.0])
    ax.set_yticklabels([])
    ax.set_xticks([0, 10, 20, 30])
    ax.set_title('isolation (km)')

    ax = fig.add_subplot(6, 6, 5, polar=True)
    h = ax.bar(np.radians(distributions['isolDir']['x']),
               distributions['isolDir']['hist'] / distributions['isolDir']['hist'].sum(), 
               width=np.diff(np.radians(distributions['isolDir']['bins'])), color=barColor, edgecolor=edgeColor, zorder=10)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_ylim([0, 0.2])
    ax.set_title('isolation dir.', pad=-3)

    ax = fig.add_subplot(6, 6, 6, polar=True)
    h = ax.bar(np.radians(distributions['saddleDir']['x']),
               distributions['saddleDir']['hist'] / distributions['saddleDir']['hist'].sum(), 
               width=np.diff(np.radians(distributions['saddleDir']['bins'])), color=barColor, edgecolor=edgeColor, zorder=10)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_ylim([0, 0.2])
    ax.set_title('saddle dir.', pad=-3)
    
    # save figure
    fig.savefig('histograms_%s.pdf' % regionName, dpi=300, bbox_inches='tight', pad_inches=0)